In [7]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

print ('Libraries imported')

Libraries imported


In [3]:
pip install html5lib

## Part 1

In [8]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
table = pd.read_html(source.text)

In [9]:
tr_df = pd.DataFrame(table[0])
tr_df.head(5)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [10]:
tr_set = {'Postal Code': 'PostalCode', 'Borough':'Borough', 'Neighborhood': 'Neighborhood'}
tr_df_temp = tr_df.rename(columns=tr_set)
tr_df_temp.head(5)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### 3.2 Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [11]:
tr_df_temp_1 = tr_df_temp[tr_df_temp['Borough'] != 'Not assigned']
tr_df_temp_1.head(10)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


## 3M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

To check for the duplication:

In [12]:
if True not in tr_df.duplicated(['Postal Code']).tolist():
    print ('No duplication')
else:
    print ('duplication exist')


No duplication


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

To check if table is empty to meet with the condition:

In [13]:
if tr_df_temp[(tr_df_temp['Neighbourhood'].isna())&(tr_df_temp['Borough']!='Not assigned')].shape[0] == 0:
    print ('Condition not met ')

Condition not met 


Final table

In [14]:
tr_df_new=tr_df_temp_1.reset_index(drop=True)
tr_df_new

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Number of rows of the dataframe:

In [15]:
tr_df_new.shape

(103, 3)

## Part 2 Geographical Coordinates

In [16]:
geo_df=pd.read_csv('Geospatial_Coordinates.csv')
geo_df.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Change the name of one column in order to have the same name in different data frame for merge-function:

In [17]:
rename_set = {'Postal Code': 'PostalCode'}
geo_df_temp = geo_df.rename(columns=rename_set)

### Then merge the data frame from part 1 and geodata:

In [18]:
df_merge = pd.merge(tr_df_new,geo_df_temp)
df_merge.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Part 3 Explore and cluster the neighborhoods in Toronto.

In [19]:
import json # library to handle JSON files
# uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge geopy --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Minee\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.9.2                |   py38haa244fe_0         3.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following packages will be UPDATED:

  conda                                4.9.0-py38h9bdc248_1 --> 4.9.2-py38haa244fe_0




conda-4.9.2          | 3.1 MB    |            |   0% 
conda-4.9.2          | 3.1 MB    |            |   1% 
conda-4.9.2          | 3.1 MB    | ####1      |  41% 
conda-4.9.2          | 3.1 MB    | #########  |  90% 
conda-4.9.2          | 3.1 MB    | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing tra

In [20]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Filter data from Part 2 to work with only boroughs that contain 'Toronto'.

In [21]:
Tor_df= df_merge[df_merge['Borough'].str.contains('Toronto')].reset_index(drop=True)
Tor_df.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


### Visualise neighbourhoods for 'Toronto'.

In [22]:
# create map of Toronto using first entries latitude and longitude values
map_tor = folium.Map(location=[Tor_df["Latitude"][0],Tor_df["Longitude"][0]], zoom_start=5)

In [24]:
# add markers to map
for lat,lng,borough,neighbourhood in zip(Tor_df['Latitude'], Tor_df['Longitude'], Tor_df['Borough'], Tor_df['Neighbourhood']):
            label = '{},{}'.format(neighbourhood,borough)
            label=folium.Popup(label, parse_html = True)
            folium.CircleMarker([lat,lng],
                               radius = 5,
                               popup=label,
                               color='blue',
                               fill=True,
                               fill_color='#3186cc',fill_opacity=0.7,parse_html=False).add_to(map_tor)

In [25]:
map_tor